In [ ]:
budget_10 = RhoZCDP(total_budget/10)
budget_90 = RhoZCDP(total_budget * 9/10)

In [ ]:
histogram = session.evaluate(
    QueryBuilder("data")
    .groupby(geo_races)
    .count(), 
    privacy_budget=budget_10
)

In [ ]:
hist_with_category = histogram.withColumn(
    "under_threshold", 
    "IF(count < 42, 'true', 'false')"
)

In [ ]:
session.create_view(
    QueryBuilder("data")
    .join_public(hist_with_category), 
    "data_with_category"
)

In [ ]:
new_sessions = session.partition_and_secret(
    "data_with_category", 
    privacy_budget=budget_90, 
    attr_name = "under_threshold", 
    splits={
        "small_groups": "true", 
        "large_groups": "false"
    }
)

In [ ]:
small_session = new _sessions["small_groups"]
small_group_counts = small_session_evaluate(
    QueryBuilder("data_with_category")
        .groupby(geo_races)
        .count(), 
    privacy_budget=budget_90
)

In [ ]:
large_session = new _sessions["large_groups"]
large_group_stats = large_session_evaluate(
    QueryBuilder("data_with_category")
        .groupby(geo_races * age_buckets)
        .count(), 
    privacy_budget=budget_90
)